In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [38]:
data = pd.read_csv("XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv")
data.dropna(inplace = True)
data.dtypes
data.head() 

,Year,Major,University,Time,Order
0,Year 2,Physics,Indiana State University,12,Fried Catfish Basket
1,Year 3,Chemistry,Ball State University,14,Sugar Cream Pie
2,Year 3,Chemistry,Butler University,12,Indiana Pork Chili
3,Year 2,Biology,Indiana State University,11,Fried Catfish Basket
4,Year 3,Business Administration,Butler University,12,Indiana Corn on the Cob (brushed with garlic b...


In [60]:
#set(data['University'])
set(data['Order']) # to see how many orders we are taking

{'Breaded Pork Tenderloin Sandwich',
 'Cornbread Hush Puppies',
 'Fried Catfish Basket',
 'Hoosier BBQ Pulled Pork Sandwich',
 'Indiana Buffalo Chicken Tacos (3 tacos)',
 'Indiana Corn on the Cob (brushed with garlic butter)',
 'Indiana Pork Chili',
 'Sugar Cream Pie',
 'Sweet Potato Fries',
 'Ultimate Grilled Cheese Sandwich (with bacon and tomato)'}

Ethical Implications:

1. Ensuring data privacy and anonymity to protect customer information.
2. Considering potential biases in the data, such as demographic biases or biases in the order history.

Business Outcome Implications:

1. Accurate predictions can lead to increased customer satisfaction and loyalty.
2. Inaccurate predictions can result in poor customer service and lost business opportunities.

Technical Implications:

1. Ensuring data quality and consistency to avoid errors in the model.
2. Ensuring the model is scalable and can handle large datasets efficiently.

3. Model Building

1. Convert the Year, Major, University, and Time columns to numerical variables.
2. Use a random forest regressor to train the model.
3. If the accuracy of the model is not satisfactory, use hyperparameter tuning with GridSearch to improve it.

In [40]:
from sklearn.model_selection import train_test_split

X = data.drop(['Order'],axis =1)
y = data['Order']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [24]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(X_train, y_train)
model.score(X_test, y_test)

In [46]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

feature = ['Order']

label_encoder = LabelEncoder()

label_encoder.fit(y)


transformed_y = label_encoder.transform(y)

transformed_y


[2 7 6 ... 7 7 0]


In [47]:
#turn the catergoris into numbers
features = ['Year','Major','University','Time']

transformer = ColumnTransformer([("one_hot",one_hot,features)], remainder = "passthrough")

transformed_X = transformer.fit_transform(X)

transformed_X



<5000x44 sparse matrix of type '<class 'numpy.float64'>'
	with 20000 stored elements in Compressed Sparse Row format>

In [90]:
X_train, X_test, y_train, y_test =  train_test_split(transformed_X,transformed_y,test_size = 0.2)

dx = pd.DataFrame(transformed_X)
dy = pd.DataFrame(transformed_y)

train_data = dx.join(dy, lsuffix='_dx', rsuffix='_dy')

model.fit(X_train,y_train)

#train_data = transformed_X.join(transformed_y)
#sns.heatmap(train_data.corr(),annot=True, cmap = "YlGnBu" )

RandomForestRegressor()

In [82]:
model.score(X_test, y_test)

0.339711755790873

In [54]:
best_forest = grid_search.best_estimator_
best_forest.score(X_test ,y_test)

0.5066983661795187

**Here we can see it performs better**

In [53]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100,200,300], 
#   "max_features": [8,12,20],
    "min_samples_split":[2,4,6,8],
    "max_depth":[None,4,8]
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5,
                           scoring="neg_mean_squared_error",
                           return_train_score=True)

grid_search.fit(transformed_X, transformed_y)  # we can change the value for the param_grid to improve the accuracy

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 4, 8], 'max_features': [8, 12, 20],
                         'min_samples_split': [2, 4, 6, 8],
                         'n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='neg_mean_squared_error')

#### I am going to build a function after this

In [94]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

def modeling_data(dataset_path):
 

    data = pd.read_csv(dataset_path)
    data = data.dropna()
    X = data.drop(['Order'],axis =1)
    y = data['Order']
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
    
    model = RandomForestRegressor()
    
    feature = ['Order']

    label_encoder = LabelEncoder()

    label_encoder.fit(y)

    transformed_y = label_encoder.transform(y)
    
    features = ['Year','Major','University','Time']

    transformer = ColumnTransformer([("one_hot",one_hot,features)], remainder = "passthrough")

    X_train, X_test, y_train, y_test =  train_test_split(transformed_X,transformed_y,test_size = 0.2)
   

    param_grid = {
    "n_estimators": [100,200,300], 
    "min_samples_split":[2,4,6,8],
    "max_depth":[None,4,8]
    }

    grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5,
                           scoring="neg_mean_squared_error",
                           return_train_score=True)

    grid_search.fit(transformed_X, transformed_y)

    best_forest = grid_search.best_estimator_

    with open("model.pkl", "wb") as f:
        pickle.dump(best_forest, f)

    return best_forest.score(X_test, y_test)


In [97]:
modeling_data("XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv")

0.3862775608120055